In [4]:
import os
import numpy as np
import tensorflow as tf
import keras

from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2 # kernel regularizer
from keras.layers import Dropout, Dense, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score

## Download dataset 
https://drive.google.com/drive/folders/1vI8Bkk5DojitLNkpz-UT30jOEay0XXon?usp=sharing

## Load data

In [5]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [ ]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [9]:
# Builde model
model = Sequential() # Sequential groups a linear stack of layers
model.add(tf.keras.Input(shape=(32, 32, 3)))
# filter 32
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same',
          kernel_regularizer=l2(5e-4), activation='relu')) # Add Another Convolution layers
model.add(keras.layers.BatchNormalization())

# filter 64
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same',
          kernel_regularizer=l2(5e-4), activation='relu')) # Add Another Convolution layers
model.add(keras.layers.BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # Add Max pooling to lower the sptail dimension
model.add(Dropout(0.25)) # reduce overfitting by dropping out some neurons

# filter 64
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same',
          kernel_regularizer=l2(5e-4), activation='relu')) # Add Another Convolution layers
model.add(keras.layers.BatchNormalization())

# filter 128
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same',
          kernel_regularizer=l2(5e-4))) # Add Another Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(keras.layers.BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # Add Max pooling to lower the sptail dimension

model.add(Flatten()) # Flatten the featuremaps
model.add(Dense(units=128)) # Add dense layer with ㄉㄚ neurons
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# augmented data
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# Setup some hyperparameters
batch_size = 64
epochs = 50
learning_rate = 1e-3

# initiate Adam optimizer
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9,
                    beta_2=0.999, epsilon=1e-4, decay=learning_rate/epochs)

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# split dataset
x_t, x_v, y_t, y_v = train_test_split(x_train, y_train, stratify=y_train, test_size=0.1)
# Fit the data into model
model.fit(x=aug.flow(x_t, y_t, batch_size=batch_size),
          epochs=epochs,
          validation_data=(x_v, y_v),
          shuffle=True)
tf.keras.models.save_model(model, 'tensorFlow.h5')


Epoch 1/50
704/704 [==============================] - 51s 36ms/step - loss: 1.8661 - accuracy: 0.3768 - val_loss: 1.9463 - val_accuracy: 0.3960
Epoch 2/50
704/704 [==============================] - 25s 36ms/step - loss: 1.4555 - accuracy: 0.5005 - val_loss: 1.5389 - val_accuracy: 0.5252
Epoch 3/50
704/704 [==============================] - 25s 36ms/step - loss: 1.2762 - accuracy: 0.5716 - val_loss: 1.1981 - val_accuracy: 0.6042
Epoch 4/50
704/704 [==============================] - 25s 36ms/step - loss: 1.1660 - accuracy: 0.6171 - val_loss: 1.0597 - val_accuracy: 0.6684
Epoch 5/50
704/704 [==============================] - 26s 36ms/step - loss: 1.0822 - accuracy: 0.6537 - val_loss: 1.3952 - val_accuracy: 0.6076
Epoch 6/50
704/704 [==============================] - 25s 36ms/step - loss: 1.0281 - accuracy: 0.6770 - val_loss: 1.1083 - val_accuracy: 0.6832
Epoch 7/50
704/704 [==============================] - 25s 35ms/step - loss: 0.9923 - accuracy: 0.6934 - val_loss: 0.9110 - val_accuracy:

In [14]:
tf.keras.models.save_model(model, 'tensorFlow2.h5')

In [ ]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [ ]:
y_pred[0] 

array([0.10074495, 0.09942016, 0.09930699, 0.09967201, 0.10006965,
       0.10010082, 0.10066631, 0.10081104, 0.09939595, 0.0998121 ],
      dtype=float32)

In [ ]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

7

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [10]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
assert y_pred.shape == (10000,)

(10000,)


In [11]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.8337


In [17]:
model = keras.models.load_model('tensorFlow.h5')
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.8337
